Calculate Metrics
=================
Calculates a number of metrics for an already existing street graph with lanes before and after

Prerequisites: Prepare the street graph and rebuild it

In [ ]:
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
outputs_path = data_directory + 'outputs/' + PERIMETER + '/'
metrics_path = data_directory + 'metrics/' + PERIMETER + '/'

Loading data
------------
Loads the street graph and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(outputs_path + 'street_graph_edges.gpkg', outputs_path + 'street_graph_nodes.gpkg')

print('Load lane geometries')
lane_geoms = snman.io.import_geofile_to_gdf(outputs_path + 'lane_geometries.shp', crs=2056)

print('Load measurement regions')
measurement_regions_gdf = snman.io.load_measurement_regions(inputs_path + 'measurement_regions/measurement_regions.gpkg',)

Calculating metrics
-------------------
This step will calculate a series of network metrics for lane graphs of different modes before and after rebuilding.
Please note that all calculations only work with strongly connected graphs. Therefore, all lane graphs will be automatically clipped only to the largest strongly connected component. Set *plot_scc=True* to show which parts of the graphs are removed (nodes are colored by their strongly connected component). Most likely, there will always be a few nodes that must be removed. But if they are many, then the results will be distorted.

For better accuracy, it's recommended that you adjust the perimeter of your network such that there are no nodes outside the largest connected component.

In [ ]:
# filter out the highways
G_filtered = snman.street_graph.filter_by_hierarchy(G, snman.hierarchy.HIERARCHIES.difference(snman.hierarchy.HIGHWAY))

In [ ]:
G_filtered = G.copy()
for uvk, data in G.edges.items():
    if data['hierarchy'] == snman.hierarchy.HIGHWAY:
        G_filtered.remove_edge(*uvk)

In [ ]:
# calculate the metrics
metrics = snman.stats.network_metrics_for_all_measurement_regions(G_filtered, measurement_regions_gdf, plot_scc=True)

Viewing the metrics
-------------------
The metrics are saved as a dict of DataFrames, one for each measurement region.
See the docustring of snman.lane_graph.calculate_stats for an explanation of the measures.

In [ ]:
metrics['zurich_city'].to_excel(metrics_path + 'metrics_zurich_city.xlsx', index=True)

In [ ]:
metrics['zurich_city']

In [ ]:
import snman.osmnx_customized as oxc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

measurement_region = measurement_regions_gdf.loc['zurich_city']

OSM_HIGHWAY_ORDER = [
    'primary', 'secondary', 'tertiary',
    'service', 'residential', 'living_street',
    'cycleway', 'footway', 'path', 'unclassified',
    '(other)'
]

plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.family"] = "Arial"

H = oxc.truncate.truncate_graph_polygon(
    G_filtered,
    measurement_region.geometry,
    quadrat_width=100, retain_all=True
)

sums = []
for ln_desc in [KEY_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER]:
    L = snman.lane_graph.create_lane_graph(H, ln_desc)
    L = snman.graph.keep_only_the_largest_connected_component(L)
    L_gdf = oxc.graph_to_gdfs(L, nodes=False)
    L_gdf['osm_highway'] = L_gdf['osm_highway'].apply(lambda x: x if x in OSM_HIGHWAY_ORDER else '(other)')
    L_gdf['osm_highway'] = pd.Categorical(L_gdf['osm_highway'], OSM_HIGHWAY_ORDER)
    L_gdf['area'] = L_gdf['length'] * L_gdf['width'] * L_gdf['twin_factor']
    sum_area_all = sum(L_gdf['area'])
    sum_area_cycling = sum(L_gdf[L_gdf['primary_mode'] == 'cycling']['area'])
    area_pct = 100 * sum_area_cycling / sum_area_all
    print(sum_area_cycling / 1000000, 'km2')
    print(sum_area_all / 1000000, 'km2')
    print(area_pct, '%')
    sums.append(
        L_gdf[L_gdf['primary_mode'] == 'cycling'].groupby(['osm_highway'])['area'].sum()/sum_area_all
    )

merged = pd.merge(*sums, left_index=True, right_index=True)
#merged['key'] = merged.index
#merged.to_excel(metrics_path + 'metrics_area.xlsx', index=True)

# Sort the dataframe in descending order
#merged = merged.sort_values(by='osm_highway', ascending=False)
merged = merged.T
merged.index = ['before', 'after']
#merged = merged.sort_values(by=merged.columns.tolist(), ascending=False)

# Create a stacked bar plot
ax = merged.plot(kind='barh', stacked=True, figsize=(6, 3), edgecolor='white', linewidth=0.3)


# Create the custom legend
#ax.legend(legend_handles, legend_colors.keys())

# Customize the plot
ax.set_xlabel('% of total road space')
ax.set_title('Stacked Bar Plot')
ax.set_xlim(0, 100)

# Add values to the stack plot
up = 0
for index, row in merged.iterrows():
    stack_right = 0
    for column in merged.columns:
        value = row[column]
        ax.annotate(
            str(round(value,1)) + '%' if value >= 2 else '',
            xy=(stack_right + value/2, up),
            ha='center', va='center', rotation=90
        )
        stack_right += value
    up += 1

ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 1, 1])

plt.savefig(metrics_path + 'plot.png')

# Show the plot
plt.show()


In [ ]:
H = oxc.truncate.truncate_graph_polygon(
    G_filtered,
    measurement_region.geometry,
    quadrat_width=100, retain_all=True
)

In [ ]:
sum(
    list(
        map(
            lambda edge: edge[1]['length'] / 1000,
            H.edges.items()
        )
    )
)

In [ ]:
lanetypes = [
    LANETYPE_MOTORIZED,
    LANETYPE_HIGHWAY,
    LANETYPE_DEDICATED_PT,
    LANETYPE_CYCLING_TRACK,
    LANETYPE_CYCLING_LANE,
    LANETYPE_CYCLING_PSEUDO,
    LANETYPE_FOOT_CYCLING_MIXED,
    LANETYPE_FOOT,
    LANETYPE_PARKING_PARALLEL,
    LANETYPE_PARKING_PERPENDICULAR,
    LANETYPE_PARKING_DIAGONAL,
    LANETYPE_NON_TRAFFIC
]

In [ ]:
L = snman.lane_graph.create_lane_graph(H, KEY_LANES_DESCRIPTION)
L = snman.graph.keep_only_the_largest_connected_component(L)
results = {}
for lanetype in lanetypes:
    results[lanetype] = round(
        sum(
            list(
                map(
                    lambda edge: edge[1]['length'] * edge[1]['twin_factor'] / 1000000
                    if edge[1]['lanetype'] == lanetype
                    else 0, 
                    L.edges.items()
                )
            )
        )
    , 3)
results

In [ ]:
round(
    sum(
        list(
            map(
                lambda edge: edge[1]['length'] / 1000,
                H.edges.items()
            )
        )
    )
, 3)

In [ ]:
lane_geoms.query(
    'lanes_key=="ln_desc_after"'
).apply(lambda row: row['length'] * row['width_m'], axis=1).sum()

In [ ]:
109297 / 245751

In [ ]:
45.78 / 8.61

In [ ]:
merged.T.to_excel(metrics_path + 'metrics_cycling_perc.xlsx', index=True)

In [ ]:
sum(list(map(lambda edge: edge[1]['length'], G.edges.items())))

In [ ]:
import shapely as shp
sum(list(map(lambda edge: shp.length(edge[1]['geometry']), G.edges.items())))